In [46]:
# Import Library
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import ddddocr
import datetime

In [47]:
# Define Parameter
url_1 = "https://www.ris.gov.tw/info-doorplate/app/doorplate/query"
url_2 = "https://www.ris.gov.tw/info-doorplate/app/doorplate/inquiry/date"

In [48]:
headers = {
          "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
          }
cookies = {"cookies":"JSESSIONID=AAF584AD4D880B5AD93FCE47280F01FE.sris-aw-info-doorplate-3; \
           _ga=GA1.3.2002745824.1682738242; \
           JSESSIONID=770A404C5B8691FBE9D205436D9923C5.sris-aw-root-2; \
           _gid=GA1.3.1716816783.1682920561"
           
          }
payloadData_map = {
            "cityCode": "68000000",
            "searchType": "date",
            }

In [49]:
# Start Crawler
session = requests.Session()
result = session.post(url=url_1, headers=headers,cookies=cookies ,data=payloadData_map)
soup = BeautifulSoup(result.text, 'html.parser')

In [50]:
# GET CAPTCHA_KEY and image
a_tags = soup.find_all(value=re.compile("^[a-zA-Z0-9]{32}$"))
for tag in a_tags:
    CAP = (tag.get('value'))
print(CAP)

51e1911538034da0a93f05f757eda4c8


In [51]:
CAP_image = f"https://www.ris.gov.tw/info-doorplate/captcha/image?CAPTCHA_KEY={CAP}&time=1682931576385.jpeg"
print(CAP_image)

https://www.ris.gov.tw/info-doorplate/captcha/image?CAPTCHA_KEY=51e1911538034da0a93f05f757eda4c8&time=1682931576385.jpeg


In [53]:
# download the CAPTCHA image
img_data = requests.get(CAP_image).content
with open('CAP_image.jpeg', 'wb') as handler:
    handler.write(img_data)

In [54]:
# CAPTCHA image Recognition
ocr = ddddocr.DdddOcr()
with open('CAP_image.jpeg', 'rb') as f:
    image_bytes = f.read()
res = ocr.classification(image_bytes)
print(res)

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
pa8q9


In [55]:
# define payload for query
payloadData = {
    "searchType": "date",
    "cityCode": 68000000,
    "tkt": -1,
    "areaCode": 68000010,
    "village": "",
    "neighbor": "",
    "sDate": "111-01-01",
    "eDate": str(datetime.date.today(),
    "_includeNoDate": "on",
    "registerKind": 0,
    "captchaInput": res,
    "captchaKey": CAP,
    "floor": "",
    "lane": "",
    "alley": "",
    "number": "",
    "number1": "",
    "ext": "",
    "_search": "false",
    "nd": 1682931576385,
    "rows": 50000,
    "page": 1,
    "sidx": "",
    "sord": "asc"
     }
result = session.post(url=url_2, headers=headers,cookies=cookies ,data=payloadData)
# session.config['keep_alive'] = False
soup = BeautifulSoup(result.text, 'html.parser')

In [56]:
# Data Clearing
Map = pd.DataFrame({ "key" : ['1','2','3','4','5','6','7'],
                    "values" : ["門牌初編","門牌改編","門牌增編","門牌合併","門牌廢止","行政區域調整","門牌整編"]})
df = pd.DataFrame(json.loads(result.text)["rows"])
df = df.merge(Map,how='left', left_on="v3", right_on="key")
df = df.rename(columns={"v1":"門牌資料","v2":"編釘日期","values":"編釘類別"})[["門牌資料","編釘日期","編釘類別"]]
df

,門牌資料,編釘日期,編釘類別
0,桃園市桃園區三元里010鄰公園路５３號,民國112年4月27日,門牌廢止
1,桃園市桃園區三元里010鄰公園路５５號,民國112年4月27日,門牌廢止
2,桃園市桃園區三元里010鄰公園路５７號,民國112年4月27日,門牌廢止
3,桃園市桃園區三元里010鄰公園路５９號,民國112年4月27日,門牌廢止
4,桃園市桃園區三元里010鄰公園路６１號,民國112年4月27日,門牌廢止
...,...,...,...
22048,桃園市桃園區中正里026鄰厦門街６６號二樓,民國111年1月6日,門牌初編
22049,桃園市桃園區中正里026鄰厦門街６６號八樓,民國111年1月6日,門牌初編
22050,桃園市桃園區中正里026鄰文中東路１６５號,民國111年1月6日,門牌初編
22051,桃園市桃園區中正里026鄰文中東路１６９號,民國111年1月6日,門牌初編


In [57]:
# Save data to csv
df.to_csv("試題1_CRAWLER.csv")